In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import numpy as np
from scipy import misc
import sys
import math
sys.path.insert(0,'../libs')
import ruidos as noise
from scipy.signal import convolve2d
%aimport edge_detectors
# import edge_detectors as ed
from image_helpers import *

In [ ]:
img = misc.imread("lena.png")
print(img.shape)

In [ ]:
# Filtro de Gradiente (con Sobel)
Sx = [[-1,0,1],[-2,0,2],[-1,0,1]]
Sy = [[1,2,1],[0,0,0],[-1,-2,-1]]

# Filtro Gaussiano
kernel_gauss = np.array([\
          [1,4,7,4,1],\
          [4,16,26,16,4],\
          [7,26,41,26,7],\
          [1,4,7,4,1],\
          [4,16,26,16,4]])*1/273

# Gradiente del kernel Gaussiano
kernel_gauss_dx = convolve2d(kernel_gauss,Sx)
kernel_gauss_dy = convolve2d(kernel_gauss,Sy)


def rec2pol(img_x,img_y):
    abs = np.empty(img_x.shape)
    ang = np.empty(img_x.shape)
    for i in range(len(img_x)):
        for j in range(len(img_x[0])):
            abs[i][j] = np.sqrt(img_x[i][j]**2+img_y[i][j]**2)
            ang[i][j] = np.arctan(img_y[i][j]/(img_x[i][j]+0.000000001)) + math.pi/2

    return abs, ang

img_x = convolve2d(img,kernel_gauss_dx)
img_y = convolve2d(img,kernel_gauss_dy)

J_m, J_o = rec2pol(img_x,img_y)


show_img_s([img,J_m,J_o],['original','Jm','Jo'])

In [ ]:
# Busco la dirección que mejor aproxima

def supresion_no_max(J_o,J_m):
    J_n = np.empty(J_m.shape)
    for i in range(1,len(J_o)-1):
        for j in range(1,len(J_o)-1):
            # Busco la direccion dk que mejor aproxima
            min = math.inf
            dk = 0
            for val in [0,math.pi/4,math.pi/2,math.pi*3/4,math.pi]:
                if abs(J_o[i][j]-val) < min:
                    min = abs(J_o[i][j]-val)
                    dk = val
                    
            # Busco sus vecinos en esa dirección
            vecino1 = 0
            vecino2 = 0
            if dk == 0 or dk == math.pi:
                vecino1 = J_m[i][j-1]
                vecino2 = J_m[i][j+1]
            elif dk == math.pi/4:
                vecino1 = J_m[i-1][j-1]
                vecino2 = J_m[i+1][j+1]                
            elif dk == math.pi/2:
                vecino1 = J_m[i-1][j]
                vecino2 = J_m[i+1][j]                
            elif dk == math.pi*3/4:
                vecino1 = J_m[i-1][j+1]
                vecino2 = J_m[i+1][j-1]  
                
            # Lo suprimo de ser necesario
            if J_m[i][j]  < vecino1 or J_m[i][j] < vecino2:
                J_n[i][j] = 0
            else:
                J_n[i][j] = J_m[i][j]
    return J_n

J_n = supresion_no_max(J_o,J_m)
show_img(J_n)


In [ ]:
# Supresión de no máximos
